In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
import sklearn.metrics as metrics

In [ ]:
global_data = pd.read_parquet('data/global_data_2023only_processed.parquet', engine='pyarrow')
na_data = pd.read_parquet('data/na_data_2023only_processed.parquet', engine='pyarrow')
na_data["date"] = pd.to_datetime(na_data["date"], format='%Y-%m-%d')
global_data["date"] = pd.to_datetime(global_data["date"], format='%Y-%m-%d')

na_data = na_data[na_data["date"] <= pd.Timestamp("2023-08-30")]
global_data = global_data[global_data["date"] <= pd.Timestamp("2023-08-30")]
merged_data = pd.merge(global_data, na_data[["date", "gvkey", "volume_usd_1"]], on=['date', 'gvkey'], how='left', suffixes=('', '_na'))
merged_data["volume_usd_1_na"] = merged_data["volume_usd_1_na"].fillna(0)

merged_data['volume_usd_1'] = merged_data['volume_usd_1_na'] + merged_data['volume_usd_1']
merged_data.drop(labels="volume_usd_1_na", axis="columns", inplace=True)

na_global_data = pd.concat([merged_data, na_data], ignore_index=True)
na_global_data.drop_duplicates(["date", "gvkey"], keep="first", inplace=True)

na_global_data.sort_values(["date", "gvkey"], inplace=True)

na_global_data_2023 = na_global_data

na_data_2023 = na_data
global_data_2023 = global_data

In [ ]:
market_cap_cutoff_quantile = 0.65

In [ ]:
volume_usd_5_min = 1000000

In [ ]:
print("Global 2023 mean market cap: ", global_data_2023["market_cap_usd"].mean())
print("Global 2023 min market cap: ", global_data_2023["market_cap_usd"].min())
print("Global 2023 mean bottom 10% market cap: ", global_data_2023["market_cap_usd"].quantile(0.05).mean())
print("Unique gvkeys: ", len(global_data_2023["gvkey"].unique()))
print("Unique gvkeys with >= 100 rows: ", global_data_2023["gvkey"].value_counts()[global_data_2023["gvkey"].value_counts() >= 100].shape[0])
global_data_2023_capped = global_data_2023.groupby("date").apply(lambda x: x[x["market_cap_usd"] > x["market_cap_usd"].quantile(market_cap_cutoff_quantile)]).reset_index(drop=True)
global_data_2023_capped = global_data_2023_capped[global_data_2023_capped["volume_usd_5"] > volume_usd_5_min]
#na_global_data_2002_capped = na_global_data_2002.groupby("date").apply(lambda x: x.nlargest(10000, "market_cap_usd")).reset_index(drop=True)
print("Global 2023 mean market cap: ", global_data_2023_capped["market_cap_usd"].mean())
print("Global 2023 min market cap: ", global_data_2023_capped["market_cap_usd"].min())
print("Global 2023 mean bottom 10% market cap: ", global_data_2023_capped["market_cap_usd"].quantile(0.05).mean())
print("Unique gvkeys: ", len(global_data_2023_capped["gvkey"].unique()))
print("Unique gvkeys with >= 100 rows: ", global_data_2023_capped["gvkey"].value_counts()[global_data_2023_capped["gvkey"].value_counts() >= 100].shape[0])

In [ ]:
market_cap_cutoff_quantile = 0.6

In [ ]:
volume_usd_5_min = 1000000

In [ ]:
print("NA 2023 mean market cap: ", na_data_2023["market_cap_usd"].mean())
print("NA 2023 min market cap: ", na_data_2023["market_cap_usd"].min())
print("NA 2023 mean bottom 10% market cap: ", na_data_2023["market_cap_usd"].quantile(0.05).mean())
print("Unique gvkeys: ", len(na_data_2023["gvkey"].unique()))
print("Unique gvkeys with >= 100 rows: ", na_data_2023["gvkey"].value_counts()[na_data_2023["gvkey"].value_counts() >= 100].shape[0])
na_data_2023_capped = na_data_2023.groupby("date").apply(lambda x: x[x["market_cap_usd"] > x["market_cap_usd"].quantile(market_cap_cutoff_quantile)]).reset_index(drop=True)
na_data_2023_capped = na_data_2023_capped[na_data_2023_capped["volume_usd_5"] > volume_usd_5_min]
#na_global_data_2002_capped = na_global_data_2002.groupby("date").apply(lambda x: x.nlargest(10000, "market_cap_usd")).reset_index(drop=True)
print("NA 2023 mean market cap: ", na_data_2023_capped["market_cap_usd"].mean())
print("NA 2023 min market cap: ", na_data_2023_capped["market_cap_usd"].min())
print("NA 2023 mean bottom 10% market cap: ", na_data_2023_capped["market_cap_usd"].quantile(0.05).mean())
print("Unique gvkeys: ", len(na_data_2023_capped["gvkey"].unique()))
print("Unique gvkeys with >= 100 rows: ", na_data_2023_capped["gvkey"].value_counts()[na_data_2023_capped["gvkey"].value_counts() >= 100].shape[0])

In [ ]:
#MERGE NA AND GLOBAL DATA
na_data_2023_capped["date"] = pd.to_datetime(na_data_2023_capped["date"], format='%Y-%m-%d')
global_data_2023_capped["date"] = pd.to_datetime(global_data_2023_capped["date"], format='%Y-%m-%d')
merged_data = pd.merge(global_data_2023_capped, na_data_2023_capped[["date", "gvkey", "volume_usd_1"]], on=['date', 'gvkey'], how='left', suffixes=('', '_na'))
merged_data["volume_usd_1_na"] = merged_data["volume_usd_1_na"].fillna(0)

merged_data['volume_usd_1'] = merged_data['volume_usd_1_na'] + merged_data['volume_usd_1']
merged_data.drop(labels="volume_usd_1_na", axis="columns", inplace=True)

na_global_data_2023_capped = pd.concat([merged_data, na_data_2023_capped], ignore_index=True)
na_global_data_2023_capped.drop_duplicates(["date", "gvkey"], keep="first", inplace=True)

na_global_data_2023_capped.sort_values(["date", "gvkey"], inplace=True)

In [ ]:
na_global_2023_n = na_global_data_2023.groupby("exchange_code").apply(lambda x: x["gvkey"].value_counts()[x["gvkey"].value_counts() >= 100].shape[0])
na_global_2023_market_cap_sum = na_global_data_2023.groupby("exchange_code")["market_cap_usd"].sum()
na_global_2023_market_cap_mean = na_global_data_2023.groupby("exchange_code")["market_cap_usd"].mean()
na_global_capped_2023_n = na_global_data_2023_capped.groupby("exchange_code").apply(lambda x: x["gvkey"].value_counts()[x["gvkey"].value_counts() >= 100].shape[0])
na_global_capped_2023_market_cap_sum = na_global_data_2023_capped.groupby("exchange_code")["market_cap_usd"].sum()
na_global_capped_2023_market_cap_mean = na_global_data_2023_capped.groupby("exchange_code")["market_cap_usd"].mean()

unique_gvkeys = pd.DataFrame({"2023": na_global_2023_n, "capped_2023": na_global_capped_2023_n,
                              "2023_market_cap_sum": na_global_2023_market_cap_sum, "2023_market_cap_mean": na_global_2023_market_cap_mean,
                              "capped_2023_market_cap_sum": na_global_capped_2023_market_cap_sum, "capped_2023_market_cap_mean": na_global_capped_2023_market_cap_mean})

In [ ]:
exch_code_name = {
    7: "Toronto Stock Exchange (TSX)",
    9: "TSX Venture Exchange (TSXV)",
    11: "New York Stock Exchange (NYSE)",
    12: "NYSE American (AMEX)",
    14: "NASDAQ Stock Exchange",
    19: "OTC Markets Group (OTC)",
    21: "CBOE BZX",
    104: "Euronext Amsterdam",
    106: "Australian Exchange (ASX)",
    107: "Athens Exchange (ATHEX)",
    115: "Berlin Stock Exchange",
    120: "Bombay Stock Exchange (BSE)",
    132: "Euronext Brussels",
    134: "Budapest Stock Exchange",
    144: "Copenhagen Stock Exchange",
    150: "Easdaq",
    151: "Swiss Exchange (SIX/SWX)",
    154: "Deutsche Börse",
    164: "Bermuda Stock Exchange",
    167: "NYSE Arca",
    170: "Hong Kong Stock Exchange (HKEX)",
    171: "Xetra (Deutsche Börse)",
    172: "Irish Stock Exchange (ISE)",
    192: "Lima Stock Exchange (BVL)",
    194: "London Stock Exchange (LSE)",
    201: "Madrid Stock Exchange (BME)",
    208: "Mexican Stock Exchange (BMV)",
    209: "Italian Stock Exchange",
    212: "Moscow Stock Exchange (MOEX)",
    219: "India Stock Exchange (NSE)",
    225: "New Zealand Exchange (NZX)",
    228: "Oslo Stock Exchange (OSE)",
    233: "Stock Exchange of Mauritius (SEM)",
    245: "Taipei Exchange (GTSM)",
    248: "Korea Exchange (KRX)",
    249: "Shanghai Stock Exchange (SSE)",
    250: "Shenzhen Stock Exchange (SZSE)",
    251: "Singapore Exchange (SGX)",
    256: "Stockholm Stock Exchange",
    260: "Taiwan Stock Exchange (TWSE)",
    264: "Tokyo Stock Exchange (TSE)",
    273: "Vienna Stock Exchange (WBAG)",
    278: "Wellington",
    280: "Zurich",
    286: "Euronext Paris",
    293: "Tokyo Stock Exchange JASDAQ",
    298: "Korea Exchange KOSDAQ",
    343: "National Stock Exchange of Australia",
    345: "The International Stock Exchange",
    353: "Bats Europe"
}

In [ ]:
unique_gvkeys.index = unique_gvkeys.index.map(exch_code_name)

In [ ]:
unique_gvkeys["2023_pct_rem"] = ((unique_gvkeys["2023"] - unique_gvkeys["capped_2023"])/unique_gvkeys["2023"])
unique_gvkeys["2023_n_rem"] = (unique_gvkeys["2023"] - unique_gvkeys["capped_2023"])
unique_gvkeys["2023_pct_mc_rem"] = ((unique_gvkeys["2023_market_cap_sum"] - unique_gvkeys["capped_2023_market_cap_sum"])/unique_gvkeys["2023_market_cap_sum"])
unique_gvkeys["2023_n_mc_rem"] = ((unique_gvkeys["2023_market_cap_sum"] - unique_gvkeys["capped_2023_market_cap_sum"]))

In [ ]:
unique_gvkeys_2023 = unique_gvkeys.sort_values(by="capped_2023", ascending=False).head(25)

unique_gvkeys_2023 = unique_gvkeys_2023.sort_values(by="capped_2023", ascending=True)[["2023", "capped_2023", "2023_pct_rem", "2023_n_rem"]].dropna()
plt = unique_gvkeys_2023.sort_values(by="capped_2023", ascending=True)[["2023", "capped_2023"]].plot(kind="barh", figsize=(10,20), grid=True)

for i in range(len(unique_gvkeys_2023)):
    plt.annotate("-" + str((unique_gvkeys_2023["2023_pct_rem"][i]*100).round(2)) + "%" + ", -" + str((int(unique_gvkeys_2023["2023_n_rem"][i]))) , (unique_gvkeys_2023["capped_2023"][i] + 50, i+0.05), rotation=0, color='red', fontsize=12)
plt.set_xlabel("Number of unique securities appearing in at least 100 days", fontsize=14)
plt.set_ylabel("")
plt.tick_params(axis='both', which='major', labelsize=12)

plt.legend(["Before market cap cutoff", "After market cap cutoff"], bbox_to_anchor=(0.55, 0.0), loc='lower center', ncol=2, fontsize=12)
#plt.figure.savefig("Exchanges2023N.pdf", dpi=1000, bbox_inches='tight')

In [ ]:
avg_daily_market_cap_per_exchange = na_global_data_2023.groupby(["exchange_code", "date"])["market_cap_usd"].sum().reset_index()
avg_daily_market_cap_per_exchange = avg_daily_market_cap_per_exchange.groupby("exchange_code")["market_cap_usd"].mean().reset_index()
avg_daily_market_cap_per_exchange_capped = na_global_data_2023_capped.groupby(["exchange_code", "date"])["market_cap_usd"].sum().reset_index()
avg_daily_market_cap_per_exchange_capped = avg_daily_market_cap_per_exchange_capped.groupby("exchange_code")["market_cap_usd"].mean().reset_index()
avg_daily_market_cap_per_exchange = avg_daily_market_cap_per_exchange.merge(avg_daily_market_cap_per_exchange_capped, on="exchange_code", suffixes=("_uncapped", "_capped"))
avg_daily_market_cap_per_exchange["pct_rem"] = ((avg_daily_market_cap_per_exchange["market_cap_usd_uncapped"] - avg_daily_market_cap_per_exchange["market_cap_usd_capped"])/avg_daily_market_cap_per_exchange["market_cap_usd_uncapped"])
avg_daily_market_cap_per_exchange["n_rem"] = (avg_daily_market_cap_per_exchange["market_cap_usd_uncapped"] - avg_daily_market_cap_per_exchange["market_cap_usd_capped"])
avg_daily_market_cap_per_exchange = avg_daily_market_cap_per_exchange.sort_values(by="market_cap_usd_uncapped", ascending=True)

avg_daily_market_cap_per_exchange = avg_daily_market_cap_per_exchange.sort_values(by="market_cap_usd_capped", ascending=False).head(25)
avg_daily_market_cap_per_exchange = avg_daily_market_cap_per_exchange.sort_values(by="market_cap_usd_capped", ascending=True)

plt = avg_daily_market_cap_per_exchange.sort_values(by="market_cap_usd_capped", ascending=True)[["market_cap_usd_uncapped", "market_cap_usd_capped"]].plot(kind="barh", figsize=(10,20), grid=True)
for i in range(len(avg_daily_market_cap_per_exchange)):
    if avg_daily_market_cap_per_exchange["pct_rem"][i] > 0:
        plt.annotate("-" + str((avg_daily_market_cap_per_exchange["pct_rem"][i]*100).round(2)) + "%" + ", -$" + str(("{:,d}".format(int(avg_daily_market_cap_per_exchange["n_rem"][i]/1000000000)))) + "B" , (avg_daily_market_cap_per_exchange["market_cap_usd_capped"][i] + 1*10**11, i+0.05), rotation=0, color='red', fontsize=12)
    else:
        plt.annotate("+" + str((abs(avg_daily_market_cap_per_exchange["pct_rem"][i]*100)).round(2)) + "%" + ", +$" + str(("{:,d}".format(int(abs(avg_daily_market_cap_per_exchange["n_rem"][i]/1000000000))))) + "B" , (avg_daily_market_cap_per_exchange["market_cap_usd_capped"][i] + 1*10**11, i+0.05), rotation=0, color='red', fontsize=12)
plt.set_xlabel("Total market cap (Billion USD)", fontsize=14)
plt.set_ylabel("")
plt.legend(["Before market cap cutoff", "After market cap cutoff"], bbox_to_anchor=(0.55, 0.0), loc='lower center', ncol=2, fontsize=12)

plt.tick_params(axis='both', which='major', labelsize=12)

plt.set_xlim(0, [avg_daily_market_cap_per_exchange["market_cap_usd_uncapped"].max() + 5.9*10**12][0])
plt.set_xticklabels(["{:,d}".format(int(x/1000000000)) for x in plt.get_xticks().tolist()])
plt.figure.savefig("Exchanges2023MC.pdf", dpi=1000, bbox_inches='tight')


In [ ]:
avg_daily_market_cap_per_currency = na_global_data_2023.groupby(["currency", "date"])["market_cap_usd"].sum().reset_index()
avg_daily_market_cap_per_currency = avg_daily_market_cap_per_currency.groupby("currency")["market_cap_usd"].mean().reset_index()
avg_daily_market_cap_per_currency_capped = na_global_data_2023_capped.groupby(["currency", "date"])["market_cap_usd"].sum().reset_index()
avg_daily_market_cap_per_currency_capped = avg_daily_market_cap_per_currency_capped.groupby("currency")["market_cap_usd"].mean().reset_index()
avg_daily_market_cap_per_currency = avg_daily_market_cap_per_currency.merge(avg_daily_market_cap_per_currency_capped, on="currency", suffixes=("_uncapped", "_capped"))
avg_daily_market_cap_per_currency["pct_rem"] = ((avg_daily_market_cap_per_currency["market_cap_usd_uncapped"] - avg_daily_market_cap_per_currency["market_cap_usd_capped"])/avg_daily_market_cap_per_currency["market_cap_usd_uncapped"])
avg_daily_market_cap_per_currency["n_rem"] = (avg_daily_market_cap_per_currency["market_cap_usd_uncapped"] - avg_daily_market_cap_per_currency["market_cap_usd_capped"])
avg_daily_market_cap_per_currency = avg_daily_market_cap_per_currency.sort_values(by="market_cap_usd_uncapped", ascending=True)

avg_daily_market_cap_per_currency.index = avg_daily_market_cap_per_currency["currency"]
avg_daily_market_cap_per_currency = avg_daily_market_cap_per_currency.sort_values(by="market_cap_usd_capped", ascending=True)

plt = avg_daily_market_cap_per_currency.sort_values(by="market_cap_usd_capped", ascending=True)[["market_cap_usd_uncapped", "market_cap_usd_capped"]].plot(kind="barh", figsize=(10,15), grid=True)
for i in range(len(avg_daily_market_cap_per_currency)):
    if avg_daily_market_cap_per_currency["pct_rem"][i] > 0:
        plt.annotate("-" + str((avg_daily_market_cap_per_currency["pct_rem"][i]*100).round(2)) + "%" + ", -$" + str(("{:,d}".format(int(avg_daily_market_cap_per_currency["n_rem"][i]/1000000000)))) + "B" , (avg_daily_market_cap_per_currency["market_cap_usd_capped"][i] + 1*10**11, i+0.05), rotation=0, color='red', fontsize=12)
    else:
        plt.annotate("+" + str((abs(avg_daily_market_cap_per_currency["pct_rem"][i]*100)).round(2)) + "%" + ", +$" + str(("{:,d}".format(int(abs(avg_daily_market_cap_per_currency["n_rem"][i]/1000000000))))) + "B" , (avg_daily_market_cap_per_currency["market_cap_usd_capped"][i] + 1*10**11, i+0.05), rotation=0, color='red', fontsize=12)
plt.set_xlabel("Total market cap (Billion USD)", fontsize=14)
plt.set_ylabel("")
plt.legend(["Before market cap cutoff", "After market cap cutoff"], bbox_to_anchor=(0.55, 0.0), loc='lower center', ncol=2, fontsize=12)
plt.set_xlim(0, [avg_daily_market_cap_per_currency["market_cap_usd_uncapped"].max() + 9.7*10**12][0])

plt.tick_params(axis='both', which='major', labelsize=12)
plt.set_xticklabels(["{:,d}".format(int(x/1000000000)) for x in plt.get_xticks().tolist()])
plt.figure.savefig("Currencies2023MC.pdf", dpi=1000, bbox_inches='tight')

In [ ]:
na_global_2023_n = na_global_data_2023.groupby("currency").apply(lambda x: x["gvkey"].value_counts()[x["gvkey"].value_counts() >= 100].shape[0])
na_global_2023_market_cap_sum = na_global_data_2023.groupby("currency")["market_cap_usd"].sum()
na_global_2023_market_cap_mean = na_global_data_2023.groupby("currency")["market_cap_usd"].mean()
na_global_capped_2023_n = na_global_data_2023_capped.groupby("currency").apply(lambda x: x["gvkey"].value_counts()[x["gvkey"].value_counts() >= 100].shape[0])
na_global_capped_2023_market_cap_sum = na_global_data_2023_capped.groupby("currency")["market_cap_usd"].sum()
na_global_capped_2023_market_cap_mean = na_global_data_2023_capped.groupby("currency")["market_cap_usd"].mean()

unique_gvkeys = pd.DataFrame({"2023": na_global_2023_n, "capped_2023": na_global_capped_2023_n, #"2002": na_global_2002_n, "capped_2002": na_global_capped_2002_n,
                              "2023_market_cap_sum": na_global_2023_market_cap_sum, "2023_market_cap_mean": na_global_2023_market_cap_mean,
                              "capped_2023_market_cap_sum": na_global_capped_2023_market_cap_sum, "capped_2023_market_cap_mean": na_global_capped_2023_market_cap_mean})

In [ ]:
unique_gvkeys["2023_pct_rem"] = ((unique_gvkeys["2023"] - unique_gvkeys["capped_2023"])/unique_gvkeys["2023"])
unique_gvkeys["2023_n_rem"] = (unique_gvkeys["2023"] - unique_gvkeys["capped_2023"])
unique_gvkeys["2023_pct_mc_rem"] = ((unique_gvkeys["2023_market_cap_sum"] - unique_gvkeys["capped_2023_market_cap_sum"])/unique_gvkeys["2023_market_cap_sum"])
unique_gvkeys["2023_n_mc_rem"] = ((unique_gvkeys["2023_market_cap_sum"] - unique_gvkeys["capped_2023_market_cap_sum"]))

In [ ]:
unique_gvkeys_2023 = unique_gvkeys.sort_values(by="capped_2023", ascending=True)[["2023", "capped_2023", "2023_pct_rem", "2023_n_rem"]].dropna()
plt = unique_gvkeys_2023.sort_values(by="capped_2023", ascending=True)[["2023", "capped_2023"]].plot(kind="barh", figsize=(10,15), grid=True)

for i in range(len(unique_gvkeys_2023)):
    plt.annotate("-" + str((unique_gvkeys_2023["2023_pct_rem"][i]*100).round(2)) + "%" + ", -" + str((int(unique_gvkeys_2023["2023_n_rem"][i]))) , (unique_gvkeys_2023["capped_2023"][i] + 50, i+0.05), rotation=0, color='red', fontsize=12)
plt.set_xlabel("Number of unique securities appearing in at least 100 days", fontsize=14)
plt.set_ylabel("")
plt.tick_params(axis='both', which='major', labelsize=12)
plt.legend(["Before market cap cutoff", "After market cap cutoff"], bbox_to_anchor=(0.55, 0.0), loc='lower center', ncol=2, fontsize=12)
plt.figure.savefig("Currencies2023N.pdf", dpi=1000, bbox_inches='tight')